# Tools for SUITE Risk-Limiting Election Audits



In [45]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from ballot_comparison import ballot_comparison_pvalue
from fishers_combination import fisher_combined_pvalue, maximize_fisher_combined_pvalue, \
     bound_fisher_fun, calculate_lambda_range    
from sprt import ballot_polling_sprt

from cryptorandom.cryptorandom import SHA256
from cryptorandom.sample import sample_by_index

# Initial sample size

Reported Votes

In [15]:
Nw1 = 2000
Nl1 = 1900
Nw2 = 20000
Nl2 = 19000

N1 = Nw1 + Nl1
N2 = Nw2 + Nl2
Nw = Nw1 + Nw2
Nl = Nl1 + Nl2
N = N1 + N2
reported_margin = Nw - Nl

print("Smallest margin (in votes):", reported_margin)
print("Diluted margin:", (Nw - Nl)/N)

Smallest margin (in votes): 1100
Diluted margin: 0.02564102564102564


Audit parameters

In [16]:
alpha = 0.1
n_ratio = N1/N # proportion of initial sample allocated to stratum 1


Expected sample sizes

In [41]:
def estimate_n(risk_limit, n_ratio):
    n = 0
    expected_pvalue = 1
    while (expected_pvalue > risk_limit) or (expected_pvalue is np.nan):
        n = n + 1000
        n1 = int(n_ratio * n)
        n2 = n - n1
        cvr_pvalue = lambda alloc: ballot_comparison_pvalue(n=n1, gamma=1.03905, o1=0, 
                                                    u1=0, o2=0, u2=0, 
                                                    reported_margin=reported_margin, N=N1, 
                                                    null_lambda=alloc)
        nocvr_pvalue = lambda alloc: ballot_polling_sprt(sample= np.array([0]*int(n2*Nl2/N2)+\
                                             [1]*int(n2*Nw2/N2)+\
                                             [np.nan]*int(n2*(N2-Nl2-Nw2)/N2)), \
                            popsize=N2, \
                            alpha=0.05,  # set this param but we don't need to use it
                            Vw=Nw2, Vl=Nl2, null_margin=(Nw2-Nl2) - alloc*reported_margin)['pvalue']
        # Crude maximizer for now
        res = bound_fisher_fun(Nw1, Nl1, N1, Nw2, Nl2, N2,
                       pvalue_funs=(cvr_pvalue, nocvr_pvalue), stepsize=0.5, plausible_lambda_range=(-3, 3))
        expected_pvalue = np.max(res['upper_bounds'])
        if (n % 10000)==0:
            print(n, expected_pvalue)
    
    return (n1, n2)

interact(estimate_n,
          risk_limit=widgets.FloatSlider(min=0,max=0.5,step=0.005,value=alpha), 
          n_ratio=widgets.FloatSlider(min=0,max=1,step=0.01,value=n_ratio)
        )

10000 0.4850348339626901


(1727, 17273)

<function __main__.estimate_n>

# Random sampling

In [43]:
seed = 12345
n1 = 1000
n2 = 15000

In [46]:
prng = SHA256(seed)
sample1 = sample_by_index(N1, n1, prng)
sample2 = sample_by_index(N2, n2, prng)

In [48]:
print("Stratum 1 sample:", sample1)

Stratum 1 sample: {2, 3, 2051, 7, 2056, 2055, 9, 2059, 2061, 2062, 16, 2064, 2067, 2068, 21, 2070, 23, 25, 28, 2076, 2086, 39, 2087, 41, 2090, 2092, 2093, 46, 2096, 52, 53, 2100, 2101, 57, 58, 2106, 2107, 61, 62, 63, 2112, 66, 78, 79, 80, 83, 85, 2136, 89, 2138, 93, 2143, 2144, 96, 2145, 2146, 102, 103, 105, 107, 2155, 110, 112, 113, 114, 2163, 2160, 122, 2173, 127, 2175, 2177, 130, 2184, 136, 2186, 137, 2185, 2189, 138, 2192, 2193, 146, 148, 2197, 150, 2200, 152, 153, 2205, 165, 166, 168, 2217, 2219, 2220, 173, 183, 184, 185, 2231, 188, 190, 2239, 2240, 2241, 194, 196, 2249, 2251, 206, 2254, 2255, 2257, 2265, 2266, 2267, 218, 222, 223, 2272, 2273, 2275, 2276, 2277, 2278, 231, 227, 2282, 2285, 241, 242, 243, 246, 248, 2298, 2299, 250, 255, 2303, 256, 2309, 261, 265, 2314, 2313, 270, 2318, 272, 273, 2322, 2324, 277, 2330, 2331, 2338, 290, 291, 293, 2346, 2348, 301, 2352, 304, 306, 2355, 310, 2359, 2358, 313, 2362, 315, 314, 2365, 317, 2366, 2364, 2363, 316, 323, 2374, 329, 2378, 333, 33

# Find ballots using ballot manifest

# Should more ballots be audited?

In [2]:
def f(x):
    return x

interact(f, x=10);

30